In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from scipy.stats import pearsonr
from sklearn.model_selection import TimeSeriesSplit
from joblib import dump
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore', category=UserWarning)


In [ ]:
# Load data
df_submission = pd.read_csv('../../outputs/submission_lgbm.csv')
df_test = pd.read_parquet('../../data/test.parquet')
df_train = pd.read_parquet('../../data/train.parquet')


In [ ]:
sub = df_submission.sort_values("ID")          # critical
sub.to_csv("check_submission_lgbm.csv", index=False)
sub

In [ ]:
print(sub.shape)
print(sub["prediction"].describe())
print(df_train['label'].describe())
assert sub["prediction"].isna().sum() == 0
assert len(sub) == len(df_test), "Row count mismatch"
assert sub["ID"].is_monotonic_increasing, "ID column has sorting issues"
assert sub["ID"].is_unique, "Duplicate IDs in submission!"
print("unique IDs:", sub["ID"].nunique())
assert set(sub["ID"]) == set(df_test.index), "ID values don't match test set!"
assert np.isfinite(sub["prediction"]).all(), "Predictions has infinite numbers"
assert sub["prediction"].isna().sum() == 0, "Found NaNs in predictions"
print("prediction var:", np.var(sub["prediction"]))
print("training dataset var:", np.var(df_train["label"]))

In [ ]:
plt.hist(sub["prediction"], bins=100)
plt.title("Submission prediction distribution")
plt.show()

In [ ]:
plt.hist(df_train["label"], bins=100)
plt.title("Training dataset target column distribution")
plt.show()